In [13]:
pwd

'/home/wsuser/work'

In [62]:
!pip install keras==2.7.0
!pip install tensorflow==2.5.0

In [18]:
from keras.preprocessing.image import ImageDataGenerator

In [19]:
train_datagen = ImageDataGenerator (rescale = 1./255, shear_range= 0.2,zoom_range= 0.2, horizontal_flip = True)

In [20]:
test_datagen =ImageDataGenerator (rescale = 1)

In [21]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='W-3VKROm4Ue50ECmN7ASNGQXep2_fOAS4A3Z9aE0ESkm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fertilizersrecommendationsystemfo-donotdelete-pr-tgbhy3rll4dl9w'
object_key = 'Dataset Plant Disease.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [22]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [23]:
pwd

'/home/wsuser/work'

In [24]:
import os
filenames=os.listdir('/home/wsuser/work/Dataset Plant Disease')

In [25]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/train_set',target_size = (128,128), batch_size = 32, class_mode = 'categorical')
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/test_set',target_size = (128,128), batch_size = 32, class_mode = 'categorical')

Found 11386 images belonging to 9 classes.
Found 3416 images belonging to 9 classes.


In [26]:
x_train.class_indices

{'Pepper,_bell___Bacterial_spot': 0,
 'Pepper,_bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___Late_blight': 3,
 'Potato___healthy': 4,
 'Tomato___Bacterial_spot': 5,
 'Tomato___Late_blight': 6,
 'Tomato___Leaf_Mold': 7,
 'Tomato___Septoria_leaf_spot': 8}

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten


In [28]:
model=Sequential()

In [29]:
model.add(Convolution2D(32,(3,3),input_shape = (128,128,3),activation = 'relu'))

In [30]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [31]:
model.add(Flatten())


In [32]:
model.add(Dense(units= 300, kernel_initializer ='uniform', activation = 'relu'))
model.add(Dense(units= 150, kernel_initializer ='uniform', activation = 'relu'))  
model.add(Dense(units= 9, activation='softmax', kernel_initializer ='random_uniform'))

In [33]:
model.compile(loss ='categorical_crossentropy',optimizer = "adam",metrics = ["accuracy"])

In [35]:
model.fit_generator(x_train, steps_per_epoch = 89,epochs = 3,validation_data = x_test, validation_steps = 27)

/tmp/wsuser/ipykernel_164/2041867713.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train, steps_per_epoch = 89,epochs = 3,validation_data = x_test, validation_steps = 27)


Epoch 1/3
89/89 [==============================] - 167s 2s/step - loss: 0.8528 - accuracy: 0.7055 - val_loss: 1355.3140 - val_accuracy: 0.2801
Epoch 2/3
89/89 [==============================] - 166s 2s/step - loss: 0.7661 - accuracy: 0.7314 - val_loss: 1081.2411 - val_accuracy: 0.3067
Epoch 3/3
89/89 [==============================] - 168s 2s/step - loss: 0.6654 - accuracy: 0.7654 - val_loss: 1600.5819 - val_accuracy: 0.1644


In [36]:
model.save("vegetable.h5")

In [38]:
!tar -zcvf disease_pred_vegetable_new.tgz vegetable.h5

vegetable.h5


In [39]:
ls -1

'Dataset Plant Disease'/
disease_pred_vegetable_new.tgz
vegetable.h5


In [40]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 11.1 MB/s eta 0:00:01


In [41]:
# Replace the credentials that you got from Watson Machine Learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"FtEtuM9zJk6s5kidhE_VBcZ9XMiOi9_mRMXfvWLCFv1l"
}
client= APIClient(wml_credentials)

In [42]:
client= APIClient(wml_credentials)

In [43]:
def guid_from_space_name (client, space_name):
    space = client.spaces.get_details()
    print(space)
    return(next (item for item in space['resources'] if item['entity']["name"] == space_name) ['metadata']['id'])

In [48]:
space_uid =  guid_from_space_name(client, 'Fertilizers Recommendation System For Disease Prediction')
print("Space UID= "+ space_uid)

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/59128f477268404b90068946f92f5c22:9b0e7097-0baf-4526-9c10-632f4aed43f5::', 'guid': '9b0e7097-0baf-4526-9c10-632f4aed43f5', 'name': 'Watson Machine Learning-j0', 'type': 'machine_learning'}], 'description': '', 'name': 'Fertilizers Recommendation System For Disease Prediction', 'scope': {'bss_account_id': '59128f477268404b90068946f92f5c22'}, 'stage': {'production': False}, 'status': {'state': 'active'}, 'storage': {'properties': {'bucket_name': '7b30ce8c-2b9d-45eb-bb82-55ad80888648', 'bucket_region': 'us-south', 'credentials': {'admin': {'access_key_id': '90ee62f9494d4dd997e361dcb586a3f6', 'api_key': 'Ce4ggdonj_u9O5htCdiZbUEGjBMqHR5ScJ7pie7TLQcK', 'secret_access_key': 'eaf1dd54455d1e6dd360d3a047da8083aaedc6091d155e73', 'service_id': 'ServiceId-db69ef8b-5df4-4688-8d49-d5722758afa8'}, 'editor': {'access_key_id': '1464da3157da471f87c91ee3521aac78', 'api_key': 'xsmnIiAvul6lZ0Thx2Ev5lfJVUC8MHuIaA_7kdbkuTD9'

In [49]:
client.set.default_space(space_uid)

'SUCCESS'

In [50]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [51]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [52]:
 ls

'Dataset Plant Disease'/   disease_pred_vegetable_new.tgz   vegetable.h5


In [53]:
model_details = client.repository.store_model(model= 'disease_pred_vegetable_new.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
    )

In [54]:
model_id = client.repository.get_model_id(model_details)

In [55]:
model_id

'a60484b0-57f1-4e69-a758-ad004b3416e3'

In [56]:
client.repository.download(model_id,"vegetable_model.tar.gz")

Successfully saved model content to file: 'vegetable_model.tar.gz'


'/home/wsuser/work/vegetable_model.tar.gz'

In [72]:
import keras
import tensorflow as tf
from keras.preprocessing import image 
import numpy as np

In [74]:
model =keras.models.load_model("vegetable.h5")

In [76]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='W-3VKROm4Ue50ECmN7ASNGQXep2_fOAS4A3Z9aE0ESkm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fertilizersrecommendationsystemfo-donotdelete-pr-tgbhy3rll4dl9w'
object_key = 'veggy.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [77]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [78]:
img=image.load_img(r'/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/test_set/Potato___healthy/potatohealthy.JPG',target_size =(128,128))

In [80]:
x = image.img_to_array(img) 
x = np.expand_dims(x,axis= 0)

In [81]:
pred = np.argmax(model.predict(x),axis=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [82]:
pred

array([8])

In [ ]:
list={'Pepper,_bell___Bacterial_spot': 0,
 'Pepper,_bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___Late_blight': 3,
 'Potato___healthy': 4,
 'Tomato___Bacterial_spot': 5,
 'Tomato___Late_blight': 6,
 'Tomato___Leaf_Mold': 7,
 'Tomato___Septoria_leaf_spot': 8}